# Intro, Imaging & Machine Learning in PyCrystEM

PyCrystEM is an open-source library for crystallographic electron microscopy. Multi-dimensional data processing tools build on the HyperSpy library.

This notebook demonstrates the use of PyCrystEM to: form so called 'virtual diffraction images' by plotting the diffracted intensity in a particular pixel in the reciprocal space images, learn the component patterns making up the data using machine learning techniques, and perform data enhancement as a pre-processing step for further analysis such as indexation and orientation mapping. 

## Change Log

22/03/18 Duncan Johnstone & Phillip Crout - Updated for ePSIC Hyperspy Workshop

08/06/17 Duncan Johnstone - Developed for Trondheim Diffraction Workshop

# Requirements

pyXem 0.5

HyperSpy 1.3

PyMatGen

## Contents

1. <a href='#loa'> Loading & Inspection</a>
2. <a href='#cal'> Alignment & Calibration</a>
3. <a href='#vdf'> Virtual Diffraction Imaging</a>
4. <a href='#ml'> Machine Learning SPED Data</a>
5. <a href='#vec'> Peak Finding & Vector Analysis</a>
6. <a href='#ori'> Indexation & Orientation Mapping</a>

Import pyXem

In [1]:
%matplotlib tk
import pyxem as pxm

import numpy as np
import pymatgen as pmg

## <a id='loa'></a> 1. Loading and Inspection

Load the SPED data acquired from the nanowire

In [2]:
dp = pxm.load('/home/hremadmin/nanowire_precession.hdf5')
dp = pxm.ElectronDiffraction(dp)

Look at what kind of object 'dp' is

In [ ]:
dp

Inspect the metadata associated with the object 'dp'

In [ ]:
dp.metadata

Set important experimental parameters using the built in function

In [ ]:
dp.set_experimental_parameters(accelerating_voltage=300.,
                               camera_length=21.,
                               scan_rotation=277.,
                               convergence_angle=0.7,
                               exposure_time=10.)

See how this changed the metadata

In [ ]:
dp.metadata

Set another metadata item and check it

In [ ]:
dp.metadata.set_item("General.title", 'GaAs Nanowire')
dp.metadata

Plot the data to inspect it

In [ ]:
dp.plot()

## <a id='cal'></a> 2. Alignment & Calibration

Apply distortion corrections to the data due to off-axis acquisition

In [ ]:
dp.apply_affine_transformation(np.array([[0.99,0,0],
                                         [0,0.69,0],
                                         [0,0,1]]))

Align the dataset based on the direct beam position

Measure known interplanar spacing to obtain calibration

In [ ]:
dps = dp.sum((0,1))
dps.plot()

In [ ]:
line = pxm.roi.Line2DROI(x1=25.2428, y1=65.7885, x2=120.297, y2=78.2273, linewidth=5.49734)
line.add_widget(dps)

In [ ]:
trace = line(dps)
trace.plot()

In [ ]:
recip_d111 = np.sqrt((3/5.6535**2))
recip_cal = recip_d111 / 11.4

Set data calibrations

In [ ]:
dp.set_diffraction_calibration(recip_cal)
dp.set_scan_calibration(10)

Plot the calibrated data

In [ ]:
dp.plot()

## <a id='vdf'></a> 3. Virtual Diffraction Imaging

Plot an interactive virtual image integrating intensity within a circular subset of pixels in the diffraction pattern

In [ ]:
roi = pxm.roi.CircleROI(cx=0.,cy=0, r_inner=0, r=0.07)
dp.plot_interactive_virtual_image(roi=roi)

Get the virtual diffraction image associated with the last integration window used interactively

In [ ]:
vdf = dp.get_virtual_image(roi)

Plot the virtual dark-field image

In [ ]:
vdf.plot()

Save the virtual dark-field image as a 32bit tif

In [ ]:
vdf.change_dtype('float32')
vdf.save('vdfeg.tif')

Plot the data with an adjustable marker indicating where to crop the scan region

In [ ]:
reg = pxm.roi.RectangularROI(left=50, top=750, right=290, bottom=990)
dp.plot()
reg.add_widget(dp)

Crop the dataset based on the region defined above

In [ ]:
dpc = reg(dp)

In [ ]:
dpc

# <a id='ml'></a> 4. Unsupervised learning

Perform singular value decomposition (SVD) of the data

In [ ]:
dpc.change_dtype('float')
dpc.decomposition(True, algorithm='svd')

Obtain a "Scree plot" by plotting the fraction of variance described by each principal component

In [ ]:
dpc.plot_explained_variance_ratio()

Plot the decomposition results and have a look at them

In [ ]:
dpc.plot_decomposition_results()

Perform non-negative matrix factorisation (NMF)

In [ ]:
dpc.decomposition(True, algorithm='nmf', output_dimension=3)

In [ ]:
dpc

Plot the NMF results

In [ ]:
dpc.plot_decomposition_results()

# Pattern enhancement & Background Subtraction

Perform a background subtraction

In [ ]:
dpb = dpc.remove_background(method='h-dome', h=0.4)

Obtain a radially integrated diffraction profile and plot it

In [ ]:
rp = dpb.get_radial_profile()
rp.plot()

In [ ]:
rp.plot()

# <a id='vec'></a> 5. Peak Finding & Vector Analysis

Interactively tune peak finding parameters

In [ ]:
dpb.find_peaks_interactive()

Perform peak finding on all diffraction patterns in data

In [ ]:
peaks = dpb.find_peaks(method='difference_of_gaussians',
                        min_sigma=1.,
                        max_sigma=6.,
                        sigma_ratio=1.6,
                        threshold=0.03,
                        overlap=1)

Check the peaks object

In [ ]:
peaks

Look at what's in the peaks object

In [ ]:
peaks.data

In [ ]:
peaks.plot_diffraction_vectors(xlim=1.5, ylim=1.5)

In [ ]:
bins = np.arange(0, 1.5, recip_cal)
ghist = peaks.get_magnitude_histogram(bins=bins)
ghist.plot()

Produce virtual diffraction contrast images for all diffraction vectors

In [ ]:
vdfs = peaks.get_vdf_images(dpc, radius=recip_cal*3)

In [ ]:
vdfs.plot()

In [ ]:
unique_peaks = peaks.get_unique_vectors(distance_threshold=0.03)

In [ ]:
vdfs = unique_peaks.get_vdf_images(dpc, radius=0.00889*3)

In [ ]:
vdfs.plot()

# <a id='ori'></a> 6. Indexation & Orientation Mapping

In [ ]:
from pyxem.utils.sim_utils import equispaced_so3_grid
from pyxem.generators.indexation_generator import ProfileIndexationGenerator, IndexationGenerator
from pyxem.utils.sim_utils import peaks_from_best_template
from pyxem.utils.plot import generate_marker_inputs_from_peaks
import hyperspy.api as hs

Build crystal structure

In [ ]:
Ga = pmg.Element("Ga")
As = pmg.Element("As")
lattice = pmg.Lattice.cubic(5.6535)

In [ ]:
structure = pmg.Structure.from_spacegroup("F23",lattice, [Ga,As], [[0, 0, 0],[0.25,0.25,0.25]])

Set up diffraction simulator

In [ ]:
edc = pxm.DiffractionGenerator(300, 5e-2)

Index peaks based on diffraction vector magnitude

In [ ]:
sim_prof = edc.calculate_profile_data(structure=structure,
                                      reciprocal_radius=1.3)

In [ ]:
sim_prof.plot(g_max=1.3)

In [ ]:
gmags = peaks.get_magnitudes()

In [ ]:
indexer = ProfileIndexationGenerator(gmags, sim_prof, mapping=False)

In [ ]:
results = indexer.index_peaks(tolerance=0.1)

In [ ]:
results

Simulate a library of 2D diffraction patterns for all symmetry inequivalent orientations

In [ ]:
diff_gen = pxm.DiffractionLibraryGenerator(edc)

rot_list = equispaced_so3_grid(90,180,90,5)
struc_lib = dict()
struc_lib["A"] = (structure,rot_list)
library = diff_gen.get_diffraction_library(struc_lib,
                                            calibration=recip_cal,
                                            reciprocal_radius=1.,
                                            half_shape=(72,72),
                                            representation='euler',
                                            with_direct_beam=False)

Find the best matching simulated diffraction pattern for every experimental pattern

In [ ]:
indexer = IndexationGenerator(dpb, library)
match_results = indexer.correlate()

Plot the position of strong peaks in the simulated data overlaying the experimental data for visual inspection

In [ ]:
match_peaks = match_results.map(peaks_from_best_template,phase=["A"],library=library,inplace=False)

mmx,mmy = generate_marker_inputs_from_peaks(match_peaks)
dpb.plot(cmap='viridis')
for mx,my in zip(mmx,mmy):
    m = hs.markers.point(x=mx,y=my,color='red',marker='x')
    dpb.add_marker(m,plot_marker=True,permanent=True)

Obtain an orientation map from the matching results

In [ ]:
cryst_map = match_results.get_crystallographic_map()
ori_map = cryst_map.get_orientation_image()
ori_map.plot()